$\newcommand{\bigO}[1]{\mathcal{O}\left(#1\right)}$

# The 8 Queens Problem

Given an $8 \times 8$ chess board, we want to find all the possible ways to fit 8 queens in it without them threatening each other (see the valid solution below). To solve this problem we are required to use 2 distinct algorithm design techniques: *Brute Force* and either *Divide & Conquer* or *Dynamic Programming*.

**Valid solution example:**

|    |  0 |  1 |  2 |  3 |  4 |  5 |  6 |  7 |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|  0 | .  | .  | .  | .  | .  | ♛  | .  |  . |
|  1 | .  | .  | .  | ♛  | .  | .  | .  |  . |
|  2 | .  | .  | .  | .  | .  | .  | ♛  |  . |
|  3 | ♛  | .  | .  | .  | .  | .  | .  |  . |
|  4 | .  | .  | .  | .  | .  | .  | .  |  ♛ |
|  5 | .  | ♛  | .  | .  | .  | .  | .  |  . |
|  6 | .  | .  | .  | .  | ♛  | .  | .  |  . |
|  7 | .  | .  | ♛  | .  | .  | .  | .  |  . |

To avoid any possible error when trying to print unicode characters, the programs will denote queens `♛` by `Q` and the `.` character is regarded as a blank space.

# Brute Force Solution

For the Brute Force solution we only need to find one valid configuration of the queens, thus we can use a randomized algorithm that chooses arbitrary and unique positions for the queens on the board  until it finds a valid solution. Since we know the 8 queens problem has at least a valid solution, this algorithm should eventually converge to one of them (there are 92 possible solutions).

## Libraries

In [1]:
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#include <unordered_set>

## Helper Functions

### Get Row

In [2]:
int row(int pos) {
    return pos / 8;
}

#### Specific Time Complexity

$$\bigO{1}.$$

#### General Time Complexity

$$\bigO{1}.$$

### Get Column

In [3]:
int column(int pos) {
    return pos % 8;
}

#### Specific Time Complexity

$$\bigO{1}.$$

#### General Time Complexity

$$\bigO{1}.$$

### Check Solution

In [4]:
bool checkSolution(std::unordered_set<int> queens) {
    std::unordered_set<int> rows;
    std::unordered_set<int> columns;
    std::unordered_set<int> diagonalsR;
    std::unordered_set<int> diagonalsL;
    
    for (auto it = queens.begin(); it != queens.end(); std::advance(it, 1)) {
        int y = row(*it);
        int x = column(*it);
        rows.insert(y);
        columns.insert(x);
        diagonalsR.insert(y - x);
        diagonalsL.insert(y + x);
    }
    
    return (rows.size() + 
            columns.size() +
            diagonalsR.size() + 
            diagonalsL.size() == 32);
}

### Specific Time Complexity

$$\bigO{22s + 20}$$

### General Time Complexity

$$\bigO{s}$$

### Print Solution

In [5]:
void printSolution(std::unordered_set<int> queens) {
    int position = 0;
    for (unsigned short int i = 0; i < 8; ++i) {
        for (unsigned short int j = 0; j < 8; ++j) {
            if (queens.find(position) != queens.end())
                printf("Q ");
            else
                printf(". ");
            ++position;
        }
        printf("\n");
    }
}

### Specific Time Complexity

$$\bigO{508}.$$

### General Time Complexity

$$\bigO{1}.$$

## Main

In [6]:
srand(time(NULL));

bool solved = false;
int iterations = 0;

// Board used to get random positions.
std::unordered_set<int> positions({
    0,  1,  2,  3,  4,  5,  6,  7,
    8,  9,  10, 11, 12, 13, 14, 15, 
    16, 17, 18, 19, 20, 21, 22, 23, 
    24, 25, 26, 27, 28, 29, 30, 31, 
    32, 33, 34, 35, 35, 36, 37, 38,
    39, 40, 41, 42, 43, 44, 45, 46, 
    47, 48, 49, 50, 51, 52, 53, 54, 
    56, 57, 58, 59, 60, 61, 62, 63
});

while (!solved) {
    ++iterations;
    std::unordered_set<int> queens;
    
    for (int i = 0; i < 8; ++i) {
        std::unordered_set<int>::iterator it = positions.begin();
        std::advance(it, rand() % positions.size());
        queens.insert(*it);
        positions.erase(it);    
    }
    
    solved = checkSolution(queens);
    if (solved) {
        printf("Valid solution found in %i iteration(s)\n", iterations);
        printSolution(queens);
    }
    else {
        // Restore set
        for (auto it = queens.begin(); it != queens.end(); std::advance(it, 1))
            positions.insert(*it);
    }
}

Valid solution found in 7726806 iteration(s)
. . Q . . . . . 
. . . . . . Q . 
. Q . . . . . . 
. . . . . . . Q 
. . . . Q . . . 
Q . . . . . . . 
. . . Q . . . . 
. . . . . Q . . 


### Specific Time Complexity

$$\bigO{391n + 519}.$$

### General Time Complexity

$$\bigO{n}.$$

# Dynamic Programming Solution

It is obvious that for this problem 2 queens cannot be placed in the same row or column, thus a simple divide and conquer approach could be to place queens on each column and memoize the row, so that we reduce the possibilities dramatically: 

* First we can put a queen on any of the 8 rows of the first column 

* Then we only have 7 valid rows for the second column

* Afterwards there are 6 rows for the third... 

* In general, for the $i$th column there are $8 - i$ possible positions to choose
    
Using the multiplication principle, this means the dynamic programming approach described has as little as $8!\approx 40 000$ possibilities to check. We can even save some time by checking if any piece is threatened through the diagonals, given the fact that this can be computed in constant time and we could use recursion to backtrack.

## Aditional Libraries

In [7]:
// Add file manipulation libraries
#include <fstream>
#include <string>

## Helper Functions

### Append Solution

In [8]:
void appendSolution(std::unordered_set<int> queens) {
    std::ofstream out;
    out.open("solutions.txt", std::ios_base::app);
    
    int position = 0;
    for (unsigned short int i = 0; i < 8; ++i) {
        for (unsigned short int j = 0; j < 8; ++j) {
            if (queens.find(position) != queens.end())
                out << "Q ";
            else
                out << ". ";
            ++position;
        }
        out << "\n";
    }
    out.close();
}

#### Specific Time Complexity

$$\bigO{513}.$$

#### General Time Complexity

$$\bigO{1}.$$

## Recursive Solver and Main Function

### Solver

In [9]:
bool solver(std::unordered_set<int> queens,
            int column,
            int row,
            std::unordered_set<int> validRows, 
            std::unordered_set<int> validRightDiagonals, 
            std::unordered_set<int> validLeftDiagonals,
            int* counter) {
    // Remove currently used row
    validRows.erase(row); 

    // Break branch if it tried to use an invalid diagonal
    bool invalidRightDiagonal = validRightDiagonals.find(column - row) == validRightDiagonals.end();
    bool invalidLeftDiagonal = validLeftDiagonals.find(column + row) == validLeftDiagonals.end();
    if (invalidRightDiagonal || invalidLeftDiagonal)
        return false;

    // Remove threatened diagonals
    validRightDiagonals.erase(column - row);
    validLeftDiagonals.erase(column + row);

    // Solution was valid for the column, we can increment it and check if we traversed the board                                    
    if (++column == 8) {
        ++counter[0];
        appendSolution(queens);
        return true;
    }

    // If we haven't traversed, branch for each valid row and try to solve the next column recursively
    for (auto rowIt = validRows.begin(); rowIt != validRows.end(); std::advance(rowIt, 1)) {
        queens.insert((*rowIt) * 8 + column);
        solver(queens, column, *rowIt, validRows, validRightDiagonals, validLeftDiagonals, counter);
        queens.erase((*rowIt) * 8 + column); 
    }
    
    return false;
}

#### Complexity Analysis

To obtain the time complexity of this function we are going to assign a weight to each line of code which corresponds to the number of instructions that are required to execute it. Then with that information, we can create a function that mimics the behaviour of this one, but that recieves an additional integer by reference that serves as an instruction counter. This method is valid, since the size of the board and the number of queens to place is fixed at 8, thus even if we check a lot of possible configurations the execution of the program will always be the same. Even though the size of the queens set and of the valid rows and diagonals varies with each call to the function, the queens array starts at a value of 1 the first time we call the function in main and it increases up to a maximum of 8, for the rows and diagonals, these start with a size of 7 and 14 respectively but they only decrease. Since insertion and deletion was done with sets to get constant time operations the program must be of constant general complexity ($\bigO{1}$). 

Solving a more general $N$ queens problem would not allow for a constant time algorithm and searching through the possible configurations even with this optimized version of the algorithm takes non-polynomial instructions in the number of queens.

In [10]:
bool solverCounter(std::unordered_set<int> queens,
            int column,
            int row,
            std::unordered_set<int> validRows, 
            std::unordered_set<int> validRightDiagonals, 
            std::unordered_set<int> validLeftDiagonals,
            int* counter,
            int* instructions) {
    // Remove currently used row
    *instructions += 2; // Weight
    validRows.erase(row); 

    // Break branch if it tried to use an invalid diagonal
    *instructions += 16;// Weight
    bool invalidRightDiagonal = validRightDiagonals.find(column - row) == validRightDiagonals.end();
    bool invalidLeftDiagonal = validLeftDiagonals.find(column + row) == validLeftDiagonals.end();
    
    ++(*instructions); // Weight
    if (invalidRightDiagonal || invalidLeftDiagonal)
        return false;

    // Remove threatened diagonals
    *instructions += 6; // Weight
    validRightDiagonals.erase(column - row);
    validLeftDiagonals.erase(column + row);

    // Solution was valid for the column, we can increment it and check if we traversed the board                                    
    *instructions += 2; // Weight
    if (++column == 8) {
        *instructions += 2; // Weight
        ++counter[0];
        *instructions += 513; // Weight
        //appendSolution(queens);
        return true;
    }

    // If we haven't traversed, branch for each valid row and try to solve the next column recursively
    *instructions += 4; // Weight
    for (auto rowIt = validRows.begin(); rowIt != validRows.end(); std::advance(rowIt, 1)) {
        *instructions += 3; // Loop Comparison Weight
        
        *instructions += 5; // Weight
        queens.insert((*rowIt) * 8 + column);
        
        solverCounter(queens, column, *rowIt, validRows, validRightDiagonals, validLeftDiagonals, counter, instructions);
        
        *instructions += 5; // Weight
        queens.erase((*rowIt) * 8 + column);
        
        *instructions += 1; // Loop Increment Weight
    }
    
    return false;
}

### Main

In [11]:
std::ifstream in("count.txt");
std::string text;
std::getline(in, text);
in.close();

if (text != "92") {
    // Reset file for safety
    std::ofstream solutions("solutions.txt");
    solutions.close();
    
    int counter = 0;
    
    // Try an initial guess for each row of column 0
    for (int row = 0; row < 8; ++row) {
        std::unordered_set<int> queens;
        std::unordered_set<int> validRows({0, 1, 2, 3, 4, 5, 6, 7});
        std::unordered_set<int> validRightDiagonals({-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7});
        std::unordered_set<int> validLeftDiagonals({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14});
        queens.insert(row);
        solver(queens, 0, row , validRows, validRightDiagonals, validLeftDiagonals, &counter);
    }
    
    printf("Found %i solutions!\n", counter);
    
    // Write number of solutions to another file
    std::ofstream out("count.txt");
    out << counter;
    out.close();
}
else
    printf("Problem already solved, check the solutions.txt file\n");

Found 92 solutions!


#### Specific Time Complexity

To obtain the specific time complexity we are going to use the same method, add a counter for instructions and increment it with the number of steps done in our model as the program progresses. Yet again we will avoid writing to the file so we will only execute the function when the condition `text != "92"` holds and avoid modifying the `count.txt` file.
 
Since this algorithm does not support a value given by a user like another number of queens to use it will always be executed equally and thus it cannot grow in size. That means its complexity is constant and therefore the number of steps can be counted by the following program

In [12]:
int instructions = 6; // Weight of commented code
/*std::ifstream in("count.txt");
std::string text;
std::getline(in, text);
in.close();*/

instructions += 1; // Weight
//if (text != "92") {
    instructions += 2; // Weight

    // Reset file for safety
    std::ofstream solutions("solutions.txt");
    
    instructions += 2; // Weight
    solutions.close();
    
    instructions += 2; // Weight
    int counter = 0;
    
    instructions += 2; // Weight
    // Try an initial guess for each row of column 0
    for (int row = 0; row < 8; ++row) {
        instructions += 1; // Comparison Weight
        
        instructions += 1; // Weight
        std::unordered_set<int> queens;
        
        instructions += 9; // Weight
        std::unordered_set<int> validRows({0, 1, 2, 3, 4, 5, 6, 7});
        
        instructions += 16; // Weight
        std::unordered_set<int> validRightDiagonals({-7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7});
        
        instructions += 16; // Weight
        std::unordered_set<int> validLeftDiagonals({0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14});
        
        instructions += 2; // Weight
        queens.insert(row);
        
        solverCounter(queens, 0, row , validRows, validRightDiagonals, validLeftDiagonals, &counter, &instructions);
    }
    
    instructions += 1; // Weight
    printf("Found %i solutions!\n", counter);
    
    instructions += 5; // Weight of file manipulation
    // Write number of solutions to another file
    /*std::ofstream out("count.txt");
    out << counter;
    out.close();*/
//}
//else
    //printf("Problem already solved, check the solutions.txt file\n");
printf("The algorithm will always execute %i instructions", instructions);

Found 92 solutions!
The algorithm will always execute 253717 instructions

As shown by the output and following from the previous argumentation, the specific complexity of the algorithm is
$$\bigO{253717}.$$

#### General Time Complexity

$$\bigO{1}.$$